In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
train_epochs = 3
batch_size = 100
test_batch_size = 100
learning_rate = 0.01
display_step = 10

In [3]:
(x_train_data, y_train_data), (x_test_data, y_test_data) = keras.datasets.mnist.load_data()

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_data, y_train_data)) \
    .map(lambda a, b: (tf.reshape(a, (28, 28, 1)) / 255, tf.one_hot(b, 10))) \
    .shuffle(1000).repeat(train_epochs).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test_data, y_test_data)) \
    .map(lambda a, b: (tf.reshape(a, (28, 28, 1)) / 255, tf.one_hot(b, 10))) \
    .shuffle(1000).repeat().batch(test_batch_size)

train_iterator = train_dataset.make_initializable_iterator()
train_next_element = train_iterator.get_next()
test_iterator = test_dataset.make_initializable_iterator()
test_next_element = test_iterator.get_next()

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

In [5]:
conv1 = tf.layers.conv2d(inputs=x, filters=32, kernel_size=[5, 5], strides=1, padding='same',
                         activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], strides=1, padding='same', activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=0.5)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.


In [6]:
pred = tf.layers.dense(inputs=dropout, units=10)

In [7]:
loss_function = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=pred)

Instructions for updating:
Use tf.cast instead.


In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [9]:
correct_pred = tf.equal(tf.argmax(y, 1), tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(train_iterator.initializer)
sess.run(test_iterator.initializer)

In [11]:
step = 0
while 1:
    try:
        xs, ys = sess.run(train_next_element)
    except tf.errors.OutOfRangeError:
        break

    sess.run([optimizer], feed_dict={x: xs, y: ys})
    step += 1
    if step % display_step == 0:
        xss, yss = sess.run(test_next_element)
        loss, acc = sess.run([loss_function, accuracy], feed_dict={x: xss, y: yss})
        print('step:{:d} loss:{:.05f} acc:{:.05f}'.format(step, loss, acc))

step:10 loss:2.19492 acc:0.39000
step:20 loss:0.89070 acc:0.77000
step:30 loss:0.70179 acc:0.82000
step:40 loss:0.45469 acc:0.87000
step:50 loss:0.33685 acc:0.89000
step:60 loss:0.21600 acc:0.91000
step:70 loss:0.10503 acc:0.96000
step:80 loss:0.31802 acc:0.87000
step:90 loss:0.29480 acc:0.91000
step:100 loss:0.15414 acc:0.94000
step:110 loss:0.14932 acc:0.96000
step:120 loss:0.37373 acc:0.88000
step:130 loss:0.19650 acc:0.96000
step:140 loss:0.31090 acc:0.93000
step:150 loss:0.06920 acc:0.98000
step:160 loss:0.12032 acc:0.96000
step:170 loss:0.11968 acc:0.96000
step:180 loss:0.08314 acc:0.98000
step:190 loss:0.11174 acc:0.94000
step:200 loss:0.40000 acc:0.91000
step:210 loss:0.08295 acc:0.98000
step:220 loss:0.11841 acc:0.95000
step:230 loss:0.19886 acc:0.95000
step:240 loss:0.24587 acc:0.94000
step:250 loss:0.11570 acc:0.97000
step:260 loss:0.08717 acc:0.97000
step:270 loss:0.11507 acc:0.96000
step:280 loss:0.14027 acc:0.93000
step:290 loss:0.06835 acc:0.96000
step:300 loss:0.06158 a

In [12]:
xss, yss = sess.run(test_next_element)
pred_y = np.argmax(sess.run(pred, {x: xss[:20]}), 1)
true_y = np.argmax(yss[:20], 1)
print('推测的数字', pred_y)
print('真实的数字', true_y)

推测的数字 [0 2 5 5 2 9 7 0 4 1 4 2 9 2 0 9 6 1 3 1]
真实的数字 [0 2 5 5 2 9 7 0 4 1 4 2 9 2 0 9 6 1 3 1]
